### Example for Jess Cervellon

In [9]:
import time
import json

import gspread
from oauth2client.service_account import ServiceAccountCredentials

import pandas as pd
pd.options.display.max_columns=None

import openai

In [10]:
# helper function for adding output to sheet
# write the results to a new tab on the google sheet
def empty_sheet(sheet):
    sheet_values = sheet.get_values()
    sheet_df = pd.DataFrame(sheet_values[1:], columns=sheet_values[0])

    # remove the current data in the sheet
    eraser = sheet_df.copy()
    for c in eraser.columns:
        eraser[c] = ""
    sheet.update([eraser.columns.values.tolist()] + eraser.values.tolist())
    return

# helper function to get gpt response
def get_gpt_response(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role":"user", "content": prompt}]
        
    )
    return response

In [11]:
# set open api key
with open("secrets.json") as f:
    secrets = json.load(f)

if key := secrets.get("open_ai_api_key_neal"):
    openai.api_key = key
    print(f"First part of key is: {key[:5]}")
else:
    print(f"Error. Key is {key}")
    


First part of key is: sk-HJ


In [12]:
gc = gspread.service_account(filename="./pod-pitch-credentials.json")
podcast_google_sheet = gc.open_by_key("11TGIZt-HsifcuNV9dZ2X-wIaLbyAer9nyA_E1nsm6lY")
podcast_google_sheet

<Spreadsheet 'Example Data' id:11TGIZt-HsifcuNV9dZ2X-wIaLbyAer9nyA_E1nsm6lY>

In [13]:
prompt_context_sheet = podcast_google_sheet.get_worksheet_by_id("1745698936").get_all_records()
# prompt_context_sheet

In [14]:
# load it into pandas
podcast_df = pd.DataFrame(prompt_context_sheet)

podcast_df['generated_subject_line'] = ""
podcast_df['generated_email_body'] = ""
podcast_df['raw_subject_response'] = ""
podcast_df['raw_email_response'] = ""

print(podcast_df.shape)
podcast_df.head(7)

(626, 6)


,Podcast Title,Podcast Description,Podcast Host,Recent Episode Title,Recent Episode Description,Email Example 1
0,Barcelona Virtual European Marketing Podcast,· · · The Barcelona Virtual European Marketing...,By Barcelona Virtual,S05 E05 - Women & e-Commerce · Gemma Cernuda ·...,"A new study shows, once again, the power women...","""Hey! \n\nBeen a lurking fan for a while - dec..."
1,Christian CEO Podcast with Kelly Baader,""" Kelly is the 'professional leader' personifi...",By Kelly Baader\n\nkellybaader.com,Marketing Strategies For Self Published Authors,#197: Are you a self-published author or about...,"""Hey! \n\nBeen a lurking fan for a while - dec..."
2,A Shark's Perspective,“A Shark’s Perspective” is the podcast with th...,By Kenneth Kinney\n\nasharksperspective.com,#375 - How to Connect CX to Loyalty,"Conversation with Patrick Henry, the President...","""Hey! \n\nBeen a lurking fan for a while - dec..."
3,Beyond Markets,“Beyond Markets” by Julius Baer is a series fe...,By Julius Baer,Digital Assets - The Bitcoin ETF Saga,What could a physically backed Exchange Traded...,"""Hey! \n\nBeen a lurking fan for a while - dec..."
4,Jagged with Jasravee : Cutting-Edge Marketing ...,“Creative thinking takes place neither inside ...,By Jasravee Kaur Chandra,Ep 109 : Peter Fader on Leveraging CLV and a C...,How do you define customer centricity? What it...,"""Hey! \n\nBeen a lurking fan for a while - dec..."
5,From The Blockchain,"“From The Blockchain"" brings you today’s most ...",By Ashley Smith,Club Memberships on Blockchain and Tech Innova...,Welcome to another episode of From the Blockch...,"""Hey! \n\nBeen a lurking fan for a while - dec..."
6,B2B Marketing & Copywriting,"""How can we increase conversions?"" From signin...",By Linda Melone,Using Humor to Boost Trust & Engagement,Why so serious? B2B businesses often shy away ...,"""Hey! \n\nBeen a lurking fan for a while - dec..."


In [15]:
client_info_sheet = podcast_google_sheet.get_worksheet_by_id(628119351).get_all_records()
client_df = pd.DataFrame(client_info_sheet)

client_row = client_df.iloc[0,:]

print(client_df.shape)
client_df.head()

(1, 5)


,Client Name,Current Role,Scraped LinkedIn Experience,client_blurb,client_interview
0,Jess Cervellon,"Vice President, Customer Experience @ Feastabl...",Experience\nFeastables logo\nVP of Customer Ex...,"Hey there! I'm Jessica Cervellon, the VP of Cu...","Jessica Cervellon \nYeah, I have four flower ..."


In [16]:
# set up loop
print(time.ctime())

raw_email_responses = []
raw_subject_responses = []
email_bodies = []
subject_lines = []
for i, row in podcast_df.iterrows():
    print(i)
    try:
        generated_email_response = ""
        generated_subject_response = ""
        email_body = ""
        subject_line = ""
        cold_email_prompt = f"""
            I need help writing a cold email to help me get onto a podcast as a guest.
            Please exclude the subject line & make sure the output isn't in parenthesis.
            The objective of this email is to spark the hosts interest, make me memorable & start a conversation.
            Follow best practices for high-converting emails & keep the email to less than 150 words.
            Write an initial outreach email to the hosts of this podcast: {row['Podcast Host']}
            For context to help personalize the email, the description of this podcast is:\n {row['Podcast Description']}\n
            Please briefly reference something from a recent episode episode so they believe we listened to it.
            The title of a recent episode is:\n{row['Recent Episode Title']}\n
            The description of this episode is:\n{row["Recent Episode Description"]}\n
            Here is an example of an outreach message I've written before:\n{row["Email Example 1"]}\n
            Use the same fun and personable tone of voice from my example outreach message.
            Here is a bit more info about me:\n\n
            My name:\n{client_row['Client Name']}\n
            My current role:\n{client_row['Current Role']}\n
            My experience (as listed on LinkedIn):\n{client_row['Scraped LinkedIn Experience']}\n
            A brief blurb about myself:\n{client_row['client_blurb']}\n
            """
            # Here is the script of a 10 minute interview I had with someone:\n{client_row['client_interview']}\n
            
        # call to gpt
        generated_email_response = get_gpt_response(cold_email_prompt)
        email_body = generated_email_response['choices'][0]['message']['content']
        time.sleep(5)
        
        subject_line_prompt = f"""
            I just wrote a cold outreach email to convince a podcast host to interview me
            on their show about my unique marketing strategies.
            Write a subject line with the objective of getting the receiver to open the email
            and keep reading. 
            Use a fun tone of voice. 
            Please do not include 'SUBJECT:' in the output. 
            Please make the output intriguing and unique. 
            Please make sure the output is at maximum 60 characters
            Here is the body of the email: \n{email_body}\n\n
            """
        
        # add a pause to avoid rate limit errors (might not be needed)
        time.sleep(5)
        
        # call to gpt for subject line
        generated_subject_response = get_gpt_response(subject_line_prompt)
        subject_line = generated_subject_response['choices'][0]['message']['content']
    except Exception as e:
        print(f"Error: {i}")
        print(str(e))


    # save results
    raw_email_responses.append(generated_email_response)
    raw_subject_responses.append(generated_subject_response)
    subject_lines.append(email_body)
    email_bodies.append(subject_line)
    
    # if i % 50 == 0:
    # print(i)
    
    # if i > 3:
    #     break


Thu Oct 26 07:04:12 2023
0
1
2
3
4
5
6
7
Error: 7
Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)
8
9
10
11
Error: 11
Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
Error: 64
Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
Error: 81
This model's maximum context length is 4097 tokens. However, your messages resulted in 14674 tokens. Please reduce the length of the messages.
82
Error: 82
Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)
83
Error: 83
This model's maximum context length is 4097 tokens. However, your messages resulted in 8921 tokens.

In [17]:
subject_lines

['Hey there!\n\nI\'ve been listening to the Barcelona Virtual European Marketing Podcast for a while now and I have to say, I\'m hooked. I recently tuned in to your episode with Gemma Cernuda on Women & e-Commerce and it was such an eye-opener. The power that women hold in online commerce is truly remarkable! \n\nSpeaking of remarkable, let me introduce myself. I\'m Jessica Cervellon, Vice President of Customer Experience at Feastables, a CPG company led by none other than "Mr. Beast". My career path has been quite a ride, from art school to various customer service roles to the travel industry. Now, I\'m delivering top-notch experiences at Feastables.\n\nOne thing I\'ve mastered along the way is creating mind-blowing two-way SMS campaigns that revolutionize SMS marketing. But it doesn\'t stop there - I\'ve also had the amazing opportunity to hand-deliver 11 Teslas to lucky winners across the country. It\'s been a truly unforgettable journey.\n\nI would love to share my experiences and

In [18]:
print(time.ctime())


Thu Oct 26 14:05:23 2023


AssertionError: 

In [19]:
podcast_df['generated_subject_line'] = subject_lines
podcast_df['generated_email_body'] = email_bodies
podcast_df['raw_subject_response'] = [str(x) for x in raw_subject_responses]
podcast_df['raw_email_response'] = [str(x) for x in raw_subject_responses]

podcast_df.to_csv("results.csv", index=False)

In [20]:
# output_sheet = podcast_google_sheet.get_worksheet(4)
output_sheet = podcast_google_sheet.get_worksheet_by_id(350710360)
output_sheet

<Worksheet 'output' id:350710360>

In [26]:
empty_sheet(output_sheet)

In [27]:
# add new data to output tab of google sheet
output_sheet.update(
    range_name="A1",
    values=[podcast_df.columns.values.tolist()] + podcast_df.values.tolist()
)

/Users/inc/.pyenv/versions/3.11.0/envs/pod-pitch-venv/lib/python3.11/site-packages/gspread/worksheet.py:1069: UserWarning: [Deprecated][in version 6.0.0]: method signature will change to: 'Worksheet.update(value = [[]], range_name=)' arguments 'range_name' and 'values' will swap, values will be mandatory of type: 'list(list(...))'
  warnings.warn(


{'spreadsheetId': '11TGIZt-HsifcuNV9dZ2X-wIaLbyAer9nyA_E1nsm6lY',
 'updatedRange': 'output!A1:J627',
 'updatedRows': 627,
 'updatedColumns': 10,
 'updatedCells': 6270}

In [ ]:
# end of main job. pause notebook
assert False

#### Scratch Code

In [ ]:
cold_email_prompt


In [ ]:

full_response = get_gpt_response(prompt=cold_email_prompt)
full_response

In [ ]:
str(full_response)

In [ ]:
email_body = full_response['choices'][0]['message']['content']
email_body

In [ ]:
subject_line_prompt = f"""
I just wrote a cold outreach email to convince a podcast host to interview me
on their show about my unique marketing strategies.
Write a subject line with the objective of getting the receiver to open the email
and keep reading. 
Use a fun tone of voice. 
Please do not include 'SUBJECT:' in the output. 
Please make the output intriguing and unique. 
Please make sure the output is at maximum 60 characters
Here is the body of the email: \n{email_body}\n\n
"""
subject_line_prompt
generated_subject = generate_email(subject_line_prompt)
generated_subject['choices'][0]['message']['content']

In [ ]:
# copied these prompts from the gsheet to make them easier to templatize
subject_line_prompt = (
    f"""
    I just wrote a cold outreach email to convince a podcast host to interview me
    on their show about my unique marketing strategies.
    Write a subject line with the objective of getting the receiver to open the email
    and keep reading. Here is the body of the email: "" Use a fun tone of voice. 
    Please do not include 'SUBJECT:' in the output. 
    Please make the output intriguing and unique. 
    Please make sure the output is at maximum 60 characters
    """
)
cold_email_prompt = (
    """
    I need help writing a cold email to help me get onto a podcast as a guest.
    Please exclude the subject line & make sure the output isn't in parenthesis.
    The objective of this email is to spark the hosts interest, make me memorable & start a conversation.
    Follow best practices for high-converting emails & keep the email to less than 150 words.
    Write an initial outreach email to the hosts of this podcast: ",C5, "
    For context to help personalize the email, the description of this podcast is: ",B5,  "
    Please briefly reference something from a recent episode episode so they believe we listened to it.
    The title of a recent episode is: ",D5, "
    The description of this episode is: ",E5, "
    Here is an example of an outreach message I've written before: ",F5, "
    Here is a bit more info about me: ",'Client biography'!F5
    """
)